<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_fcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the modules used
import numpy as np
import scipy as sci
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
# For reproducible results
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [ ]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [ ]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [ ]:
df = fj_load_df_from_url()
df = fj_txt_only(df)

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [ ]:
"""
Use Count Vectorizer to convert text into bag of words
"""
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

cv = CountVectorizer(strip_accents='unicode', lowercase=True, stop_words='english', dtype=np.int8) 
cv_train_sparse = cv.fit_transform(train_text)
cv_train_dense = sci.sparse.csr_matrix.todense(cv_train_sparse)

cv_test_sparse = cv.transform(test_text)
cv_test_dense = sci.sparse.csr_matrix.todense(cv_test_sparse)

print('BOW for cv_train:', cv_train_dense.shape)
print('BOW for cv_test:', cv_test_dense.shape)

BOW for cv_train: (15198, 96150)
BOW for cv_test: (2682, 96150)


In [ ]:
"""
Fully connected NN model with two hidden layers 
"""
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_dense.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy', tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               9615100   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 9,616,121
Trainable params: 9,616,121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(cv_train_dense, train_labels, epochs = 5)

Epoch 1/5
475/475 [==============================] - 33s 69ms/step - loss: 0.0867 - accuracy: 0.9715 - false_positives: 49.0000 - false_negatives: 384.0000
Epoch 2/5
475/475 [==============================] - 33s 69ms/step - loss: 0.0204 - accuracy: 0.9934 - false_positives: 35.0000 - false_negatives: 65.0000
Epoch 3/5
475/475 [==============================] - 33s 69ms/step - loss: 0.0055 - accuracy: 0.9982 - false_positives: 10.0000 - false_negatives: 17.0000
Epoch 4/5
475/475 [==============================] - 33s 69ms/step - loss: 0.0033 - accuracy: 0.9987 - false_positives: 11.0000 - false_negatives: 8.0000
Epoch 5/5
475/475 [==============================] - 33s 69ms/step - loss: 0.0015 - accuracy: 0.9997 - false_positives: 2.0000 - false_negatives: 2.0000


In [ ]:
pred_soft = model.predict(cv_test_dense)

In [ ]:
# pred = np.around(pred_soft, decimals = 0)
pred = np.where(pred_soft > 0.50, 1, 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)

cm = confusion_matrix(test_labels, pred)
tn = cm[0][0]
fn = cm[1][0]
tp = cm[1][1]
fp = cm[0][1]

print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))
print('False Positives: {:.0f}'.format(fp), 'False Negatives: {:.0f}'.format(fn))
print('Confusion matrix:\n', cm)

Accuracy score: 0.9888 F1 score: 0.8864
False Positives: 3 False Negatives: 27
Confusion matrix:
 [[2535    3]
 [  27  117]]


In [ ]:
"""
# Uncomment to save results on drive to a csv file
df_results1 = pd.DataFrame(data=test_labels)
df_results1.reset_index(drop=True, inplace=True)
df_results2 = pd.DataFrame(data=pred_soft, columns=['fcnn'])
df_results = pd.concat([df_results1, df_results2], axis=1)

from google.colab import drive
drive.mount('/content/drive')
results_file='/content/drive/My Drive/Results/fcnn.csv'

df_results.to_csv(results_file)
"""

"\n# Uncomment to save results on drive to a csv file\ndf_results1 = pd.DataFrame(data=test_labels)\ndf_results1.reset_index(drop=True, inplace=True)\ndf_results2 = pd.DataFrame(data=pred_soft, columns=['fcnn'])\ndf_results = pd.concat([df_results1, df_results2], axis=1)\n\nfrom google.colab import drive\ndrive.mount('/content/drive')\nresults_file='/content/drive/My Drive/Results/fcnn.csv'\n\ndf_results.to_csv(results_file)\n"